# Homework 3: Data Frames and Graphs  (50 points)

Please complete this notebook by filling in the cells provided. The solved homework should be uploaded to Canvas.




**Reading**: 

Data8 textbook chapters [5](https://inferentialthinking.com/chapters/05/Sequences.html) and [7.1-7.2](https://inferentialthinking.com/chapters/07/1/Visualizing_Categorical_Distributions.html).

**Deadline**:

This assignment is due on Thursday, October 20 at 9:00am. Late work will not be accepted unless you have made special arrangements with the instructors.

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged.

You should start early so that you have time to get help if you're stuck.

In [ ]:
# Don't change this cell; just run it. 

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## 1. CTA (20 points)

This exercise will look at a dataset from the Chicago Transit Authority, `CTA - Ridership - Daily Boarding Totals`, available from the Chicago Data Portal 
https://data.cityofchicago.org/Transportation/CTA-Ridership-Daily-Boarding-Totals/6iiy-9s97 
and downloaded on October 4, 2022

This is a 20-year timeseries of total bus and train utilization. The readme file on this data is here:
https://data.cityofchicago.org/api/assets/5175CFDE-CA9A-498C-BB1C-8A86B780CCF1

In [ ]:
# specify the correct path to the file and run this
cta= pd.read_csv("CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
cta.head(5)

In [ ]:
# run this
type(cta.iloc[0,0])

As you see in the output of the above cell, the `service_date` entries are strings. The next cell converts the (string) dates into pandas.datetime format and assigns them to a new column called `Date`. We also show you how to extract the year from the `Date` column (you might need to extract other parts of the date so read the code carefully).

In [ ]:
# run this
cta['Date']=pd.to_datetime(cta.service_date)
cta['Year'] = cta['Date'].dt.year
cta.tail(5)

The `datetime` data format has other useful features with some more illustrated in the cell below for the last date in the dataset.


In [ ]:
# run this
[cta.Date[7943].year,cta.Date[7943].month, cta.Date[7943].day, cta.Date[7943].dayofweek]

**Question 1**. What period of time does the dataset cover (dates are ordered)? Calculate the number of days in that period. 

**Question 2** How many rows does the dataset have?  Are any dates missing?


*Answer here:*

**Question 3** Generate a line plot of CTA total_rides for the **past five years** (since 01/01/20217).

In [ ]:
plt.xlim((pd.to_datetime("2017-01-01") , pd.to_datetime("2023-01-01")))

**Question 4** Compare total CTA ridership for the months of January in 2019 and 2022.
By approximately how much has ridership fallen from 2019 to 2022? 

**Question 5** Make a scatter plot of train boardings vs. bus ridership.  Comment on the graph.  (What do you see?)  Make sure you add adequate labels. 

**Question 6** Are there unusual days in the next plot (code provided below)? For the unusual days you find, provide an explanation.
                                                                                          

*Answer here:*

## 2. Colleges (15 points)


This exercise uses data on colleges from Homework 1. Recall that the dataset was posted on Kaggle at the beginning of 2018, and was grabbed from US-News: https://www.usnews.com. It includes statistics for 311 US Universities. 

To make your tasks easiear, we removed many of the columns and kept only 11. The corresponding file can be found here:

http://www.stat.uchicago.edu/~nicolae/118/colleges1.csv


#### Run the next cell that contains code that will allow you to read the data into a data frame called `colleges_df`. It also shows the first 10 rows.


In [ ]:
# Just run this cell.

colleges_df=pd.read_csv('http://www.stat.uchicago.edu/~nicolae/118/colleges1.csv')
colleges_df.head(10)

**Run the following command that will make a scatterplot of SAT average scores (x-axis) and tuition (y-axis).**

In [ ]:
colleges_df.plot.scatter(x='sat-avg',y='tuition');

**Question 1.**  Describe in few words the pattern you see. What is a possible explanation for this pattern?

*Write your answer here, replacing this text.*

**Question 2.** Draw a scatterplot of SAT average (x-axis) and 'cost-after-aid' (y-axis) and then describe the pattern you see. Provide an explanation for this pattern.

In [ ]:
# use this cell for the code that will draw the scatterplot

**Question 3.** Construct a data frame that contains the colleges for which 'percent-receiving-aid' is greater than 0. Call this data frame `aid_df`.

Note that this dataset has missing data points (for example, not all colleges report the percentage of students who receive aid), and the table you created has eliminated the colleges with no information on aid.

In [ ]:
aid_df=...



**Question 4.**  Calculate the proportion of students who receive aid in private universities. Similarly calculate the
proportion of students who receive aid in public universities. (Hint: the answer will not come from directly averaging  the 'percent-receiving-aid' column)

In [ ]:
# use this cell to write the code that will output the two numbers



**Question 5.** Construct a data frame with two rows (for private and public) and two columns: (i) first column should contain the strings 'public' and 'private'; (ii) second column should contain the percent of students receiving aid that you calculated in Question 4.

In [ ]:
# code for constructing the data frame

**Question 6.** Make a bar chart that shows the aid information in the table you created for  Question 5.

**Note:** Graphs aren't very useful without accurate labels.  

In [ ]:
# the bar chart

## 3. Seattle Public Library Circulation (15 points)
For this exercise we use a dataset, dating to 2005, of titles checked out at the Seattle Public Library. Since this dataset is voluminous, we have taken a random sample of 1/1000 of the data without replacement.   This dataset is distrubted from the Seattle Open Data Portal: 
https://data.seattle.gov/Community/Checkouts-by-Title/tmmm-ytt6

You can read about the origin of the dataset Walsh, Melanie "Where is all the Book Data?"  2022-10-04  https://www.publicbooks.org/where-is-all-the-book-data/ at publicbooks.org.

In [ ]:
spl_df= pd.read_csv("http://www.stat.uchicago.edu/~nicolae/118/Checkouts.sample.csv")
spl_df.head(5)

**Question 1**  Make a bar chart of the total number of items checked out by year (here we count number of items, so the entry in second row is counted once). Comment on the values in 2005, 2020, and 2022.

**Question 2** How did the pandemic interact with long-term trends in library / media 
consumption?  Using the data in the column `UsageClass` make a line graph plot that has two line trends, one for "DIGITAL" and one for "PHYSICAL". Comment on the trends you see in the graph. 	

*Answer here:* 

**Question 3** We would like to see if there is some clarification on the trends in Question 2 that come from the data in column `MaterialType`. Make a line graph of BOOK, AUDIOBOOK, EBOOK, and VIDEODISK checkouts and comment on the general trends in the graph.

*Answer here:*